In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
from mnist import MNIST

config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
session = tf.Session(config=config)

In [2]:
data = MNIST(data_dir="data/MNIST")

In [3]:
# 定义基本的全局参数

# 图像在每个维度上的像素值
img_size = data.img_size

# 图片展平后的像素值 img_size_flat
img_size_flat = data.img_size_flat

# 图像所包含的类别数
num_classes = data.num_classes

# 图像包含的通道数
num_channels = data.num_channels

In [4]:
# 分割训练集和测试集和验证集
x_train = data.x_train
y_train = data.y_train

x_test = data.x_test
y_test = data.y_test

print("train data shape:", x_train.shape, "test data shape:", x_test.shape)

train data shape: (55000, 784) test data shape: (10000, 784)


In [5]:
# 定义网络所需的输入输出变量
x = tf.placeholder(dtype=tf.float32, shape=[None, img_size_flat], name="x")
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
y_true = tf.placeholder(dtype=tf.float32, shape=[None, num_classes], name="y_true")

In [6]:
# 定义两个卷积层
# layer1：对28*28进行5*5的卷积 步长为1
# 将layer1进行池化
# layer2：对layer1输出的层进行5*5卷积 步长为2
# 将layer2进行池化
# 对池化层进行展平、进行dropout
# 建立一个dense128 的隐层
# 建立一个分类层

In [7]:
tf.nn.conv2d??

In [8]:
batch_size = 64

filter_size1 = 5

output_channels1 = 16
# conv1
# 定义卷积层
# 定义卷积测weight参数 [filter_height, filter_width, input_channel, output_channel]
weight = tf.Variable(tf.truncated_normal(shape=[filter_size1, filter_size1, num_channels, output_channels1], stddev=0.05))
# 定义卷积层的bias参数
bias = tf.Variable(tf.constant(0.05, shape=[output_channels1]))
conv = tf.nn.conv2d(
    input=x_image,
    filter=weight,
    strides=[1, 1, 1, 1],
    padding="SAME",
    name="layer1"
)
# 建模
layer1 = tf.nn.relu(conv + bias)
# 对第一层进行max_pooling
max_pool1 = tf.nn.max_pool(layer1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [9]:
tf.nn.max_pool??

In [10]:
# conv2
filter_size2 = 5

output_channels2 = 36

weight2 = tf.Variable(tf.truncated_normal(shape=[filter_size2, filter_size2, output_channels1, output_channels2], stddev=0.05))
# 定义卷积层的bias参数
bias = tf.Variable(tf.constant(0.05, shape=[output_channels2]))

conv2 = tf.nn.conv2d(
    input=max_pool1,
    filter=weight2,
    strides=[1, 1, 1, 1],
    padding='SAME'
)

layer2 = tf.nn.relu(conv2 + bias)

max_pool2 = tf.nn.max_pool(layer2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [11]:
max_pool2.shape

TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(36)])

In [12]:
max_pool1.shape

TensorShape([Dimension(None), Dimension(14), Dimension(14), Dimension(16)])

In [13]:
# 对layer.get_shape()函数掌握不清楚这里需要借助该函数进行推到
# layer.get_shape().num_features() 可以计算该层的参数个数
conv_out_shape = max_pool2.get_shape()
num_features = conv_out_shape[1:4].num_elements()

In [14]:
# 将最后的卷积输出展平 然后进行池化
flatten = tf.reshape(max_pool2, shape=[-1, num_features])

In [15]:
output_layer1 = tf.nn.dropout(flatten, keep_prob=0.7)

In [16]:
flatten

<tf.Tensor 'Reshape_1:0' shape=(?, 1764) dtype=float32>

In [17]:
# 后接一个dense 128 的全连接层
units_num = 128
weight_dense1 = tf.Variable(tf.random_normal(shape=[num_features, units_num]))
bias_dense1 = tf.constant(0.05, shape=[units_num])
dense1 = tf.nn.relu(tf.matmul(output_layer1, weight_dense1) + bias_dense1)

In [18]:
# 后接类别数为num_classes的分类器
weight_dense2 = tf.Variable(tf.random_normal(shape=[units_num, num_classes], stddev=0.05))
bias_dense2 = tf.constant(0.05, shape=[num_classes])
logits = tf.matmul(dense1, weight_dense2) + bias_dense2

In [19]:
logits.shape

TensorShape([Dimension(None), Dimension(10)])

In [20]:
# 对输出的logits进行 softmax变换得出预测结果
# 对预测结果和真实值进行对比衡量两个概率分布之间的差距 即求出cross_entropy
# tf 提供了一个可以直接进行softmax激活并求出entropy
entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_true)

In [21]:
loss = tf.reduce_mean(entropy)

In [22]:
# 定义优化器 对loss进行优化 
optimizer = tf.train.AdamOptimizer(learning_rate=2e-4).minimize(loss)

In [23]:
# 定义打印准确率的函数
prediction = tf.nn.softmax(logits)
# 对打印正确率的做法不清晰（numpy知识比较匮乏）
# tf.argmax 返回的是指定维度最大元素的索引
pred_cls = tf.argmax(prediction, axis=1)

correct_prediction = tf.equal(pred_cls, tf.argmax(y_true, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [24]:
# 搭建计算流图
batch_size = 32
num_iteration = 200
total_iteration = 0
session = tf.Session()

init = tf.global_variables_initializer()
session.run(init)

In [25]:
total_iteration = 0
def optimize(num_iteration=200):
    global total_iteration
    for i in range(total_iteration, total_iteration + num_iteration):
        x_batch, y_batch, y_cls_batch = data.random_batch(batch_size)
        feed_train_data = {x:x_batch, y_true: y_batch}
        session.run(optimizer, feed_dict=feed_train_data)
        if i % 100 == 0:
            acc = session.run(accuracy, feed_dict=feed_train_data)
            print("total_iteration:", total_iteration, "acc:", acc)
    total_iteration += num_iteration

In [26]:
optimize(7000)

total_iteration: 0 acc: 0.125
total_iteration: 0 acc: 0.40625
total_iteration: 0 acc: 0.71875
total_iteration: 0 acc: 0.6875
total_iteration: 0 acc: 0.6875
total_iteration: 0 acc: 0.875
total_iteration: 0 acc: 0.90625
total_iteration: 0 acc: 0.9375
total_iteration: 0 acc: 0.875
total_iteration: 0 acc: 0.9375
total_iteration: 0 acc: 0.9375
total_iteration: 0 acc: 1.0
total_iteration: 0 acc: 0.8125
total_iteration: 0 acc: 0.84375
total_iteration: 0 acc: 0.90625
total_iteration: 0 acc: 0.875
total_iteration: 0 acc: 0.90625
total_iteration: 0 acc: 0.9375
total_iteration: 0 acc: 0.9375
total_iteration: 0 acc: 0.96875
total_iteration: 0 acc: 0.96875
total_iteration: 0 acc: 0.9375
total_iteration: 0 acc: 0.96875
total_iteration: 0 acc: 0.90625
total_iteration: 0 acc: 0.9375
total_iteration: 0 acc: 0.96875
total_iteration: 0 acc: 0.90625
total_iteration: 0 acc: 0.96875
total_iteration: 0 acc: 0.96875
total_iteration: 0 acc: 0.96875
total_iteration: 0 acc: 0.9375
total_iteration: 0 acc: 0.96875

In [ ]:
# 总结：
#1. numpy函数和基本的操作需要加强
#2. tf 变量初始化需要加强
# 3.卷积层和池化层初始化参数理解